In [2]:
import itertools
import pandas as pd
import scipy.interpolate as inter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.signal as sig
import scipy.misc as misc
import scipy.integrate as integrate
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import erfc

def mintime(time):
    return 5*np.ceil(time/5.0)

def maxtime(time):
    return 5*np.floor(time/5.0)

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = sig.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = sig.filtfilt(b, a, data)
    return y

def partial_derivative2d(function, x, y, dx, dy):
    value1 = function(x-dx/2, y-dy/2)
    value2 = function(x+dx/2, y+dy/2)
    return (value2-value1)/np.sqrt(dx**2+dy**2)
    
def CloughTocher2d_interpolator(xref, yref, vals):
    """ 2d interpolation (requires Scipy).
     
    This is a convenience wrapper around
    scipy.interpolation.CloughTocher2Dinterpolator, that saves you
    having to worry about using meshgrid.
 
    Parameters
    ----------
    xref, yref : array of floats, shapes (J,), (I,)
      Reference coordinate grid. The grid must be equally spaced along
      each direction, but the spacing can be different between
      directions.
    vals : array of floats, shape (I, J)
      Reference values at the reference grid positions.
 
    Returns
    -------
    interpolator: CloughTocher2DInterpolater instance
      Object that accepts a (y,x) tuple (note reversed order from the
      input to this function!) and returns the interpolated value.
 
    See Also
    --------
    barak.plot.arrplot for plotting the reference and interpolated arrays.
    """
 
    assert (len(yref), len(xref)) == vals.shape
    XREF,YREF = np.meshgrid(xref, yref)
    
    interpolator = inter.CloughTocher2DInterpolator((XREF.ravel(), YREF.ravel()),
                                              vals.ravel())
 
    return interpolator

def constantfluxsurface(x, t, params):
    F0 = params[0]
    k = params[1]
    kappa = params[2]
    temps = 2*F0/2*((kappa*t/np.pi)**0.5*np.exp(-x/(4*kappa*t))-x/2*erfc(x/(2*kappa *t)**0.5))
    return temps

def residual(x, t, params, measured_temp):
    calculated_temps = consantfluxsurface(x, t, params)
    return temps - calculated_temps
    
    

    
order = 3
fs = 10.0   # sample rate, Hz
cutoff = 0.025# desired cutoff frequency of the filter, Hz



In [97]:
# load all the data into the script and plot a key temperature to decide the start and end of the heat flux experiment

filenames = ['Test1_1_rep1','Test1_2_rep1','Test1_3_rep1','Test1_4_rep1','Test1_5_rep1','Trial1', 'Trial2']



dfs = []

test_TC = "T26"
i = 0
for filename in filenames:
    df = pd.read_feather("Processed//"+filename+'.feather')
    dfs.append(df)
    plt.plot(df["time"],df[test_TC], label = i)
    i = i+1
plt.legend()
plt.show()

The above plot is used to ascertain the start and end of each of the heat flux experiments, this data is then inputted in the "start_ends" list below which crops the data to this range

In [100]:
i=0
start_ends = [[10,250],[0,240],[0,240],[5,340],[10,400],[105,340],[65,305]]
for i in range(len(dfs)):
    dfs[i]['time'] = dfs[i]['time'] - start_ends[i][0]
    dfs[i] = dfs[i][dfs[i]['time']<start_ends[i][1]-start_ends[i][0]]
    dfs[i] = dfs[i][dfs[i]['time']>=0]
    dfs[i] = dfs[i].sort_index()

The flux data is calculated using a function copied from the master data processing script on github, flux and HTC columns are added to the dataframes

In [102]:
# copied from master processing script

import pandas as pd
import scipy.interpolate as inter
import scipy.integrate as integrate

def heat_flux_sensor_calculation(df, sensor_information, boundary_TC):

    for TC in HeatFluxTCs[1]:
        df[TC+' change'] = df[TC] - df.iloc[0][TC]    
    perspex_energy = []
    for time in df.index:
        temp_interp = inter.UnivariateSpline(HeatFluxTCs[0], [df.at[time, x+' change'] for x in HeatFluxTCs[1]], s=0)
        perspex_energy.append(1190*1466*integrate.quad(temp_interp, a=0, b=0.02)[0])
    df['perspex energy'] = perspex_energy
    df['ali energy'] = df[HeatFluxTCs[1][0]+' change']*2710.0*910.0*0.002
    df['total energy'] = df['perspex energy']+df['ali energy']
    df['flux'] = df['total energy'].diff(1)/5.0
    df['HTC'] = df['flux']/(df[boundary_TC] - df[HeatFluxTCs[1][0]])
    return df


sensor_information = [[0.0, 0.005, 0.01, 0.015, 0.02], ['T19', 'T18', 'T20', 'T16', 'T17']]
#                      location of TC in mm from the surface, name of TC
boundary_TC = 'T23'# TC in air boundary layer
for df in dfs:
    df = heat_flux_sensor_calculation(df, sensor_information, boundary_TC)  

The flux data is then fitted to a function of flux,this can be editted in the "surface_flux" function.

Once fitted a "calc flux" column is added to the dataframe

Additionally a results list is created which stores the output from the fitting function

In [103]:
def surface_flux(params, df, HeatFluxTCs, boundary_TC):
    #convective_flux = params[0]*(df[boundary_TC]-df[HeatFluxTCs[1][0]])
    #radiative_flux = params[1]*5.670373*10**-8*(params[2]**4.0-df[HeatFluxTCs[1][0]]**4.0)
    #return convective_flux+radiative_flux
    return params[0]

def residual(params, df, HeatFluxTCs, boundary_TC, flux):
    calc_fluxes = surface_flux(params, df, HeatFluxTCs, boundary_TC)
    #print(params, df, HeatFluxTCs, boundary_TC, flux)
    return np.sum((calc_fluxes - df[flux])**2.0)

def fitting_wrapper(df, HeatFluxTCs, boundary_TC, flux):
    x0 = [3000]
    result = opt.minimize(residual, x0, args=(df, HeatFluxTCs, boundary_TC, flux), bounds=bounds)
    return result

bounds = [[0, 5000]]

results = []
for df in dfs:
    result = fitting_wrapper(df, HeatFluxTCs, boundary_TC, 'flux')
    df['calc flux'] = surface_flux(result.x, df, HeatFluxTCs, boundary_TC)
    results.append(result)

Plotting for comparing calculated to measured heat fluxes

In [104]:
%matplotlib qt

colors = itertools.cycle(cm.rainbow(np.linspace(0, 1, len(dfs))))
i = 0
for df in dfs:
    c = next(colors)
    res = results[i].x
    #label_string = f' HTC:{res[0]:.2f}, epsilon:{res[1]:.2f}, BB temp:{res[2]:.2f})'
    label_string = f' HTC:{res[0]:.2f}'
    plt.plot(df['time'], df['flux'], label = filenames[i]+ label_string, color = c, ls='-')
    plt.plot(df["time"], df['calc flux'], color = c, ls= '--')
    i = i+1
plt.legend()
plt.xlabel('Time /s')
plt.ylabel('Surface Flux /W/m2')
plt.xlim(0,400)
#plt.ylim(0,2200)
plt.grid()
plt.show()

In [59]:
%matplotlib qt
time_steps = [5.0,25.0,50.0,75.0,150.0]
colors = itertools.cycle(cm.rainbow(np.linspace(0, 1, len(time_steps))))
df = concat_dfs[0]
x_grid = np.linspace(0,0.02,100)
k = 0.21
density = 1260
Cp = 1440
kappa = k/(density*Cp)
for time_step in time_steps:
    c = next(colors)
    plt.plot(HeatFluxTCs[0], [df.at[time_step, HeatFluxTCs[1][x]+' change'] for x in [0,1,2,3,4]], color=c, marker='x', ls='None')
    #temp_interp = inter.UnivariateSpline(HeatFluxTCs[0], [df.at[time_step, x+' change'] for x in HeatFluxTCs[1]], s=0)
    #temp_interp = inter.interp1d(HeatFluxTCs[0], [df.at[time_step, x+' change'] for x in HeatFluxTCs[1]], kind='linear')
    #calc_temps = temp_interp(x_grid)
    #plt.plot(x_grid, calc_temps, color=c, ls='-')
    temps = 2*1500/k*((kappa*time_step/np.pi)**0.5*np.exp(-x_grid**2/(4*kappa*time_step)) - 
                                           x_grid/2*erfc(x_grid/(2*(kappa*time_step)**0.5)))+0   
    plt.plot(x_grid, temps, color=c, ls='-')

In [50]:
results[0]

      fun: 71508484.097418
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 8
      nit: 2
     njev: 4
   status: 0
  success: True
        x: array([2831.08136885])

In [28]:
len(concat_dfs[0].index)

68

,index,time,T0,T0RAW,T1,T1RAW,T2,T2RAW,T3,T3RAW,...,T27RAW,T27,T28RAW,T28,T29RAW,T29,T30RAW,T30,T31RAW,T31
2,1.584370e+09,0.0,0,0,0,0,0,0,0,0,...,21.560389,21.346550,22.192773,21.892752,22.000000,21.999941,22.000000,21.968852,22.000000,21.973464
3,1.584370e+09,5.0,0,0,0,0,0,0,0,0,...,21.674629,21.568279,22.250000,22.245134,22.000000,21.999969,22.000000,21.974804,22.000000,21.975896
4,1.584370e+09,10.0,0,0,0,0,0,0,0,0,...,21.785509,22.206298,22.289965,23.085017,22.000000,21.999990,22.000000,21.987309,22.000000,21.982341
5,1.584370e+09,15.0,0,0,0,0,0,0,0,0,...,23.016422,23.317164,23.976567,24.416135,22.000000,21.999986,22.000000,21.999193,22.000000,21.985867
6,1.584370e+09,20.0,0,0,0,0,0,0,0,0,...,25.031749,24.851463,26.687253,26.098619,22.000000,21.999941,22.000000,22.005984,22.000000,21.984070
7,1.584370e+09,25.0,0,0,0,0,0,0,0,0,...,26.871660,26.671773,28.292816,27.899780,22.000000,21.999865,22.000000,22.006990,21.963638,21.979416
8,1.584370e+09,30.0,0,0,0,0,0,0,0,0,...,28.810850,28.607616,29.840092,29.587841,22.000000,21.999795,22.000000,22.004277,21.941133,21.976410
9,1.584370e+09,35.0,0,0,0,0,0,0,0,0,...,30.571445,30.516732,30.782849,31.015157,22.000000,21.999787,22.000000,22.000725,22.000000,21.978026
10,1.584370e+09,40.0,0,0,0,0,0,0,0,0,...,32.237924,32.322492,32.053702,32.150946,22.000000,21.999886,22.000000,21.998391,22.000000,21.984020
11,1.584370e+09,45.0,0,0,0,0,0,0,0,0,...,33.868834,34.015502,32.887893,33.059156,22.000000,22.000085,22.000000,21.997841,22.000000,21.991852
